# Исследование надежности заемщиков


Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd # Иимпорт библиотеки pandas

data = pd.read_csv('/datasets/data.csv') # чтение файла


In [2]:
data.head(20) # просмотр первых 20 страк данных

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info() #просмотр основной информации датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Удаление пропусков

In [4]:
data.isna().sum() #вывод количества пропущенных значений для каждого столбца

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце буду медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаю значения в этом столбце: заменив все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
# Для каждого типа занятости выведите медианное значение трудового стажа в днях.
data.groupby('income_type')['days_employed'].agg('median') 

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Если понадобится вернемсяч к ним позже

In [8]:
data['children'].unique() # Вывод уникальных значений в столбце children

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удаю строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Далее Заполняем пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum() # Проверка отсутствия пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
#изменение типа данных столбца с вещественного на целочисленный
data['total_income'] = data['total_income'].astype(int) 

### Обработка дубликатов

In [14]:
data.duplicated().sum() # Просмотр количества дубликатов

54

In [15]:
data = data.drop_duplicates() # Удаление дубликатов

**Обработка неявных дубликатов в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Нужно Привести их к нижнему регистру**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique() # Просмотр уникальных значений

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создаю функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследую данные для ответов на вопросы

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
data['debt'].unique() # Проверяем уникальные значения столбца debt

array([0, 1])

*Увидев результат можно предположить что **0** - отсутствие задолжности, а **1** имел задолжность по возврату кредита*

**Создаем переменную - DataFrame, в которой сгрупируем данные по количеству детей**

In [23]:
debt_children = pd.DataFrame() # имя таблицы

In [24]:
debt_children_count = data.groupby('children')['debt'].count() # количество заемщиков сгруппированные по количеству детей 
debt_children_sum = data.groupby('children')['debt'].sum() # количество заемщиков, которые имел задолжность сгруппированных 
#по количеству детей


In [25]:
debt_children['debt_children_count'] = debt_children_count # внесение данных в таблицу в столбец debt_children_count
debt_children['debt_children_sum'] = debt_children_sum # внесение данных в таблицу в столбец debt_children_sum

In [26]:
#создаем столбец с  конверсией имеющих задолжность к общему числу заемщик в каждой группе
debt_children['debt_children_sum_convers'] = debt_children['debt_children_sum'] / debt_children['debt_children_count'] 

**Выведем таблицу и отсортируем её по возрастанию конверсии**

In [27]:
debt_children.sort_values('debt_children_sum_convers')

,debt_children_count,debt_children_sum,debt_children_sum_convers
children,,,
5,9,0,0.000000
0,14107,1063,0.075353
3,330,27,0.081818
1,4809,444,0.092327
2,2052,194,0.094542
4,41,4,0.097561


**Вывод:** 

- Бездетные заемщики более благополучные, чем с детьми
- У Заемщиков с 1-4 детьми % выплаты долга практически одинаковый, хотя заемщики с 3 детьми чаще выплачивают заем в срок
- Заемщик с 5 детьми не имеют проблем с выплатой кредита - это лучший результат, но скорей всего связанный с небольшим числом данных, которые мы имеем

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [28]:
data['family_status'].unique() #Проверяем уникальные значения в столбце famili_status

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

**Создаем переменную DataFrame, в которой будем сохранять сгруппированные переменные**

In [29]:
debt_family = pd.DataFrame()

In [30]:
debt_family_count = data.groupby('family_status')['debt'].count() # количество заемщиков сгруппированных по статусу
debt_family_sum = data.groupby('family_status')['debt'].sum() # количество заемщиков, которые имели заолжность 
#и сгруппированные по статусу

In [31]:
debt_family['debt_family_count'] = debt_family_count
debt_family['debt_family_sum'] = debt_family_sum
# создание столбцов и внесение данных

In [32]:
debt_family['debt_family_convers'] = debt_family['debt_family_sum'] / debt_family['debt_family_count'] 
# создаем столбец с конверсией имеющих задолжность к общему числу в категории

**Выведем таблицу и отсортируем её по возрастанию конверсии**

In [33]:
debt_family.sort_values(by='debt_family_convers')
# Ваш код будет здесь. Вы можете создавать новые ячейки.

,debt_family_count,debt_family_sum,debt_family_convers
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12266,927,0.075575
гражданский брак,4146,385,0.092861
Не женат / не замужем,2796,273,0.097639


**Вывод:** 

- `Не женатые/не замужем` заемщики самые не надежные по возврату кредитов. 
- `Вдовцы`  или `в разводе` чаще всего возвращают заем
- `Женатые/замужем` заемщики  и заемщики в гражданском браке имеют разный % возврата кредитных средств 
- Самый высокий процент невозвратов у `не женатых / не замужник`

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

**Так как у всех заемщиков сумма дохода очень разная - воспользуемся категориями, на которые мы разделили заемщиков в предыдущих заданиях и сгруппируем их по категориям**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Создаем переменную - DataFrame, в которой будем сохранять сгруппированные данные**

In [34]:
debt_income = pd.DataFrame()

In [35]:
debt_income_count = data.groupby('total_income_category')['debt'].count() #количество заемщиков разделенных по категориям
debt_income_sum = data.groupby('total_income_category')['debt'].sum()#количество земщиков имеющих задолжность в выплате кредита 
#по категориям

In [36]:
debt_income['debt_income_count'] = debt_income_count
debt_income['debt_income_sum'] = debt_income_sum

In [37]:
debt_income['debt_income_convers'] = debt_income['debt_income_sum'] / debt_income['debt_income_count']
#создаем столбец с  конверсией имеющих задолжность к общему числу заемщик в каждой группе

**Выведем таблицу и отсортируем её по возрастанию конверсии**

In [38]:
debt_income.sort_values(by='debt_income_convers')

,debt_income_count,debt_income_sum,debt_income_convers
total_income_category,,,
D,349,21,0.060172
B,5014,354,0.070602
A,25,2,0.080000
C,15938,1353,0.084891
E,22,2,0.090909


Из представленных данных видно что они не показательные, так как разброс в количестве заемщиков по категориям очень большой.
<br> Для лучшего результата можно обьединить категории, в более крупные.
Так как у нас есть средний показатель заемщиков с доходом до 200000(категория С), предложу поделить на 2 категории 
- Доход ниже среднего: до 200000 
- Доход выше среднего: от 200000


**Создадим функцию, которая разделит заемщиков на более узкие категории и применим её к нашей основной таблице, что бы перепроверить теорию**

In [39]:
def func_debt_income(income):
    try:

        if income <= 200000:
            return 'Ниже среднего'
        elif income >= 200001:
            return 'Выше среднего'
    
    except:
        pass
    

In [40]:
data['total_income_category_2'] = data['total_income'].apply(func_debt_income)

**Повторим предыдущие действия**

In [41]:
debt_income = pd.DataFrame()

In [42]:
debt_income_count = data.groupby('total_income_category_2')['debt'].count() #количество заемщиков разделенных по категориям
debt_income_sum = data.groupby('total_income_category_2')['debt'].sum()#количество земщиков имеющих задолжность в выплате кредита 
#по категориям

In [43]:
debt_income['debt_income_count'] = debt_income_count
debt_income['debt_income_sum'] = debt_income_sum

In [44]:
debt_income['debt_income_convers'] = debt_income['debt_income_sum'] / debt_income['debt_income_count']

In [45]:
debt_income.sort_values(by='debt_income_convers')

,debt_income_count,debt_income_sum,debt_income_convers
total_income_category_2,,,
Выше среднего,5039,356,0.070649
Ниже среднего,16309,1376,0.084371


**Вывод:** 


Обьединив малочисленные категории с более крупными заметно, что заемщики с доходом ниже среднего(менее 200 000) чаще встречатся с задолжностью в выплате кредита, чем заемщики с доходом выше среднего(более 200 000)

**Как разные цели кредита влияют на его возврат в срок?**

**Для данного исследования воспользуемся категориями целей, на которые мы ранее разбили заемщиков**

**Создаем переменную - DataFrame, в которой будем сохранять сгруппированные данные**

In [46]:
debt_purpose = pd.DataFrame()

In [47]:
debt_purpose_count = data.groupby('purpose_category')['debt'].count() #количество заемщиков разделенных по категориям
debt_purpose_sum = data.groupby('purpose_category')['debt'].sum()#количество земщиков имеющих задолжность в выплате кредита 
#по категориям

In [48]:
debt_purpose['debt_purpose_count'] = debt_purpose_count
debt_purpose['debt_purpose_sum'] = debt_purpose_sum

In [49]:
debt_purpose['debt_purpose_convers'] = debt_purpose['debt_purpose_sum'] / debt_purpose['debt_purpose_count']
#создаем столбец с  конверсией имеющих задолжность к общему числу заемщик в каждой группе

**Выведем таблицу и отсортируем её по возрастанию конверсии**

In [50]:
debt_purpose.sort_values(by='debt_purpose_convers')

,debt_purpose_count,debt_purpose_sum,debt_purpose_convers
purpose_category,,,
операции с недвижимостью,10754,780,0.072531
проведение свадьбы,2324,183,0.078744
получение образования,3989,369,0.092504
операции с автомобилем,4281,400,0.093436


**Вывод:** 

Мы видим что:
- Чаще всего проблемы с задолжностью у тех, кто берет займ для автомобиля или на образование
- Займ на операции с недвижимостью выплачиваются чаще всего
- Займ на проведение свадеб выплачивается почти так же как и займ на недвижимость

**Возможные причины появления пропусков в исходных данных.**

*Ответ:* Так как в исходных данных пропуски встречались только в столбцах со стажем и параллельно им в столбце с уровнем дохода. Очевидней всего заемщик не хотел или не мог указать или подтвердить свой доход и рабочий статус 

**Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* Так как медианное значение выдает середину ряда данных, которые отсортированные по возрастанию на две равные части и берет значение именно серидины этого ряда данных(если количество значений не четное) или среднее значение двух показателей посередине деленное на 2(для четного количества данных). 
<br>Тогда как если использовать среднее значение, то на него может повлиять вбросы данных, которые искозят искомое значение в какую либо сторону( на подобии анекдота: я ем капусту, а мой друг ест мясо, но в среднем мы оба едим голубцы)

## Общий вывод.

- Проведя ряд вычислений мы увидели, что заём на крупные покупки и события чаще возвращают, чем на более мелкие.
- Заемщики без детей реже просрочивают оплату оплату, чем больше детей, тем больше шанс просрочки платежа
- Семейный статус тоже очень показателен для заемщика, так овдовевшие заемщики или разведенные чаще выплачивают заём, а люди в гражданском браке имеют больше шансов на проблемы с просрочкой, чем женатые люди. А самый большой процент не возврата у неженатых.
- И конечно заемщики с более крупным доходом чаще выплачивают займ
